In [1]:
import os
import numpy as np

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

from test_helper import Test
import os.path

fileName = os.path.join('D:/', 'DM_ALUNO.CSV')

numPartitions = 100
AlunoRawData = sc.textFile(fileName, numPartitions, 8)

In [2]:
# pega amostra e mostra tela
numPoints = AlunoRawData.count()
print numPoints
samplePoints = AlunoRawData.take(1)
print samplePoints

# começa limpeza dos dados
from pyspark.mllib.regression import LabeledPoint
import numpy as np


header = AlunoRawData.take(1)[0] # o cabeçalho é a primeira linha do arquivo
print header

11449223
[u'CO_IES|NO_IES|CO_CATEGORIA_ADMINISTRATIVA|DS_CATEGORIA_ADMINISTRATIVA|CO_ORGANIZACAO_ACADEMICA|DS_ORGANIZACAO_ACADEMICA|CO_CURSO|NO_CURSO|CO_CURSO_POLO|CO_TURNO_ALUNO|DS_TURNO_ALUNO|CO_GRAU_ACADEMICO|DS_GRAU_ACADEMICO|CO_MODALIDADE_ENSINO|DS_MODALIDADE_ENSINO|CO_NIVEL_ACADEMICO|DS_NIVEL_ACADEMICO|CO_OCDE|NO_OCDE|CO_OCDE_AREA_GERAL|NO_OCDE_AREA_GERAL|CO_OCDE_AREA_ESPECIFICA|NO_OCDE_AREA_ESPECIFICA|CO_OCDE_AREA_DETALHADA|NO_OCDE_AREA_DETALHADA|CO_ALUNO_CURSO|CO_ALUNO_CURSO_ORIGEM|CO_ALUNO|CO_COR_RACA_ALUNO|DS_COR_RACA_ALUNO|IN_SEXO_ALUNO|DS_SEXO_ALUNO|NU_ANO_ALUNO_NASC|NU_MES_ALUNO_NASC|NU_DIA_ALUNO_NASC|NU_IDADE_ALUNO|CO_NACIONALIDADE_ALUNO|DS_NACIONALIDADE_ALUNO|CO_PAIS_ORIGEM_ALUNO|CO_UF_NASCIMENTO|CO_MUNICIPIO_NASCIMENTO|IN_ALUNO_DEF_TGD_SUPER|IN_DEF_AUDITIVA|IN_DEF_FISICA|IN_DEF_INTELECTUAL|IN_DEF_MULTIPLA|IN_DEF_SURDEZ|IN_DEF_SURDOCEGUEIRA|IN_DEF_BAIXA_VISAO|IN_DEF_CEGUEIRA|IN_DEF_SUPERDOTACAO|IN_TGD_AUTISMO_INFANTIL|IN_TGD_SINDROME_ASPERGER|IN_TGD_SINDROME_RETT|IN_TGD_

In [3]:
import re
import datetime
from collections import namedtuple

headeritems = header.split('|') # transformar o cabeçalho em lista
print headeritems

[u'CO_IES', u'NO_IES', u'CO_CATEGORIA_ADMINISTRATIVA', u'DS_CATEGORIA_ADMINISTRATIVA', u'CO_ORGANIZACAO_ACADEMICA', u'DS_ORGANIZACAO_ACADEMICA', u'CO_CURSO', u'NO_CURSO', u'CO_CURSO_POLO', u'CO_TURNO_ALUNO', u'DS_TURNO_ALUNO', u'CO_GRAU_ACADEMICO', u'DS_GRAU_ACADEMICO', u'CO_MODALIDADE_ENSINO', u'DS_MODALIDADE_ENSINO', u'CO_NIVEL_ACADEMICO', u'DS_NIVEL_ACADEMICO', u'CO_OCDE', u'NO_OCDE', u'CO_OCDE_AREA_GERAL', u'NO_OCDE_AREA_GERAL', u'CO_OCDE_AREA_ESPECIFICA', u'NO_OCDE_AREA_ESPECIFICA', u'CO_OCDE_AREA_DETALHADA', u'NO_OCDE_AREA_DETALHADA', u'CO_ALUNO_CURSO', u'CO_ALUNO_CURSO_ORIGEM', u'CO_ALUNO', u'CO_COR_RACA_ALUNO', u'DS_COR_RACA_ALUNO', u'IN_SEXO_ALUNO', u'DS_SEXO_ALUNO', u'NU_ANO_ALUNO_NASC', u'NU_MES_ALUNO_NASC', u'NU_DIA_ALUNO_NASC', u'NU_IDADE_ALUNO', u'CO_NACIONALIDADE_ALUNO', u'DS_NACIONALIDADE_ALUNO', u'CO_PAIS_ORIGEM_ALUNO', u'CO_UF_NASCIMENTO', u'CO_MUNICIPIO_NASCIMENTO', u'IN_ALUNO_DEF_TGD_SUPER', u'IN_DEF_AUDITIVA', u'IN_DEF_FISICA', u'IN_DEF_INTELECTUAL', u'IN_DEF_MULTI

In [4]:
Aluno = namedtuple('Aluno',headeritems) # gera a namedtuple Aluno com os campos de header
print Aluno._fields

('CO_IES', 'NO_IES', 'CO_CATEGORIA_ADMINISTRATIVA', 'DS_CATEGORIA_ADMINISTRATIVA', 'CO_ORGANIZACAO_ACADEMICA', 'DS_ORGANIZACAO_ACADEMICA', 'CO_CURSO', 'NO_CURSO', 'CO_CURSO_POLO', 'CO_TURNO_ALUNO', 'DS_TURNO_ALUNO', 'CO_GRAU_ACADEMICO', 'DS_GRAU_ACADEMICO', 'CO_MODALIDADE_ENSINO', 'DS_MODALIDADE_ENSINO', 'CO_NIVEL_ACADEMICO', 'DS_NIVEL_ACADEMICO', 'CO_OCDE', 'NO_OCDE', 'CO_OCDE_AREA_GERAL', 'NO_OCDE_AREA_GERAL', 'CO_OCDE_AREA_ESPECIFICA', 'NO_OCDE_AREA_ESPECIFICA', 'CO_OCDE_AREA_DETALHADA', 'NO_OCDE_AREA_DETALHADA', 'CO_ALUNO_CURSO', 'CO_ALUNO_CURSO_ORIGEM', 'CO_ALUNO', 'CO_COR_RACA_ALUNO', 'DS_COR_RACA_ALUNO', 'IN_SEXO_ALUNO', 'DS_SEXO_ALUNO', 'NU_ANO_ALUNO_NASC', 'NU_MES_ALUNO_NASC', 'NU_DIA_ALUNO_NASC', 'NU_IDADE_ALUNO', 'CO_NACIONALIDADE_ALUNO', 'DS_NACIONALIDADE_ALUNO', 'CO_PAIS_ORIGEM_ALUNO', 'CO_UF_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'IN_ALUNO_DEF_TGD_SUPER', 'IN_DEF_AUDITIVA', 'IN_DEF_FISICA', 'IN_DEF_INTELECTUAL', 'IN_DEF_MULTIPLA', 'IN_DEF_SURDEZ', 'IN_DEF_SURDOCEGUEIRA',

In [5]:
def ParseAluno(rec):
    # utilizando re.split() vamos capturar nossos valores
    CO_IES, NO_IES, CO_CATEGORIA_ADMINISTRATIVA, DS_CATEGORIA_ADMINISTRATIVA, CO_ORGANIZACAO_ACADEMICA, DS_ORGANIZACAO_ACADEMICA, CO_CURSO, NO_CURSO, CO_CURSO_POLO, CO_TURNO_ALUNO, DS_TURNO_ALUNO, CO_GRAU_ACADEMICO, DS_GRAU_ACADEMICO, CO_MODALIDADE_ENSINO, DS_MODALIDADE_ENSINO, CO_NIVEL_ACADEMICO, DS_NIVEL_ACADEMICO, CO_OCDE, NO_OCDE, CO_OCDE_AREA_GERAL, NO_OCDE_AREA_GERAL, CO_OCDE_AREA_ESPECIFICA, NO_OCDE_AREA_ESPECIFICA, CO_OCDE_AREA_DETALHADA, NO_OCDE_AREA_DETALHADA, CO_ALUNO_CURSO, CO_ALUNO_CURSO_ORIGEM, CO_ALUNO, CO_COR_RACA_ALUNO, DS_COR_RACA_ALUNO, IN_SEXO_ALUNO, DS_SEXO_ALUNO, NU_ANO_ALUNO_NASC, NU_MES_ALUNO_NASC, NU_DIA_ALUNO_NASC, NU_IDADE_ALUNO, CO_NACIONALIDADE_ALUNO, DS_NACIONALIDADE_ALUNO, CO_PAIS_ORIGEM_ALUNO, CO_UF_NASCIMENTO, CO_MUNICIPIO_NASCIMENTO, IN_ALUNO_DEF_TGD_SUPER, IN_DEF_AUDITIVA, IN_DEF_FISICA, IN_DEF_INTELECTUAL, IN_DEF_MULTIPLA, IN_DEF_SURDEZ, IN_DEF_SURDOCEGUEIRA, IN_DEF_BAIXA_VISAO, IN_DEF_CEGUEIRA, IN_DEF_SUPERDOTACAO, IN_TGD_AUTISMO_INFANTIL, IN_TGD_SINDROME_ASPERGER, IN_TGD_SINDROME_RETT, IN_TGD_TRANSTOR_DESINTEGRATIVO, CO_ALUNO_SITUACAO, DS_ALUNO_SITUACAO, QT_CARGA_HORARIA_TOTAL, QT_CARGA_HORARIA_INTEG, DT_INGRESSO_CURSO, IN_ING_VESTIBULAR, IN_ING_ENEM, IN_ING_AVALIACAO_SERIADA, IN_ING_SELECAO_SIMPLIFICADA, IN_ING_SELECAO_VAGA_REMANESC, IN_ING_SELECAO_VAGA_PROG_ESPEC, IN_ING_TRANSF_EXOFFICIO, IN_ING_DECISAO_JUDICIAL, IN_ING_CONVENIO_PECG, IN_RESERVA_VAGAS, IN_RESERVA_ETNICO, IN_RESERVA_DEFICIENCIA, IN_RESERVA_ENSINO_PUBLICO, IN_RESERVA_RENDA_FAMILIAR, IN_RESERVA_OUTRA, IN_FINANC_ESTUDANTIL, IN_FIN_REEMB_FIES, IN_FIN_REEMB_ESTADUAL, IN_FIN_REEMB_MUNICIPAL, IN_FIN_REEMB_PROG_IES, IN_FIN_REEMB_ENT_EXTERNA, IN_FIN_REEMB_OUTRA, IN_FIN_NAOREEMB_PROUNI_INTEGR, IN_FIN_NAOREEMB_PROUNI_PARCIAL, IN_FIN_NAOREEMB_ESTADUAL, IN_FIN_NAOREEMB_MUNICIPAL, IN_FIN_NAOREEMB_PROG_IES, IN_FIN_NAOREEMB_ENT_EXTERNA, IN_FIN_NAOREEMB_OUTRA, IN_APOIO_SOCIAL, IN_APOIO_ALIMENTACAO, IN_APOIO_BOLSA_PERMANENCIA, IN_APOIO_BOLSA_TRABALHO, IN_APOIO_MATERIAL_DIDATICO, IN_APOIO_MORADIA, IN_APOIO_TRANSPORTE, IN_ATIVIDADE_EXTRACURRICULAR, IN_COMPL_ESTAGIO, IN_COMPL_EXTENSAO, IN_COMPL_MONITORIA, IN_COMPL_PESQUISA, IN_BOLSA_ESTAGIO, IN_BOLSA_EXTENSAO, IN_BOLSA_MONITORIA, IN_BOLSA_PESQUISA, CO_TIPO_ESCOLA_ENS_MEDIO, IN_ALUNO_PARFOR, CO_SEMESTRE_CONCLUSAO, CO_SEMESTRE_REFERENCIA, IN_MOBILIDADE_ACADEMICA, CO_MOBILIDADE_ACADEMICA, CO_MOBILIDADE_ACADEMICA_INTERN, CO_IES_DESTINO, CO_PAIS_DESTINO, IN_MATRICULA, IN_CONCLUINTE, IN_INGRESSO_TOTAL, IN_INGRESSO_VAGA_NOVA, ANO_INGRESSO = tuple(rec.split('|'))
    

    return Aluno(CO_IES, NO_IES, CO_CATEGORIA_ADMINISTRATIVA, DS_CATEGORIA_ADMINISTRATIVA, CO_ORGANIZACAO_ACADEMICA, DS_ORGANIZACAO_ACADEMICA, CO_CURSO, NO_CURSO, CO_CURSO_POLO, CO_TURNO_ALUNO, DS_TURNO_ALUNO, CO_GRAU_ACADEMICO, DS_GRAU_ACADEMICO, CO_MODALIDADE_ENSINO, DS_MODALIDADE_ENSINO, CO_NIVEL_ACADEMICO, DS_NIVEL_ACADEMICO, CO_OCDE, NO_OCDE, CO_OCDE_AREA_GERAL, NO_OCDE_AREA_GERAL, CO_OCDE_AREA_ESPECIFICA, NO_OCDE_AREA_ESPECIFICA, CO_OCDE_AREA_DETALHADA, NO_OCDE_AREA_DETALHADA, CO_ALUNO_CURSO, CO_ALUNO_CURSO_ORIGEM, CO_ALUNO, CO_COR_RACA_ALUNO, DS_COR_RACA_ALUNO, IN_SEXO_ALUNO, DS_SEXO_ALUNO, NU_ANO_ALUNO_NASC, NU_MES_ALUNO_NASC, NU_DIA_ALUNO_NASC, NU_IDADE_ALUNO, CO_NACIONALIDADE_ALUNO, DS_NACIONALIDADE_ALUNO, CO_PAIS_ORIGEM_ALUNO, CO_UF_NASCIMENTO, CO_MUNICIPIO_NASCIMENTO, IN_ALUNO_DEF_TGD_SUPER, IN_DEF_AUDITIVA, IN_DEF_FISICA, IN_DEF_INTELECTUAL, IN_DEF_MULTIPLA, IN_DEF_SURDEZ, IN_DEF_SURDOCEGUEIRA, IN_DEF_BAIXA_VISAO, IN_DEF_CEGUEIRA, IN_DEF_SUPERDOTACAO, IN_TGD_AUTISMO_INFANTIL, IN_TGD_SINDROME_ASPERGER, IN_TGD_SINDROME_RETT, IN_TGD_TRANSTOR_DESINTEGRATIVO, CO_ALUNO_SITUACAO, DS_ALUNO_SITUACAO, QT_CARGA_HORARIA_TOTAL, QT_CARGA_HORARIA_INTEG, DT_INGRESSO_CURSO, IN_ING_VESTIBULAR, IN_ING_ENEM, IN_ING_AVALIACAO_SERIADA, IN_ING_SELECAO_SIMPLIFICADA, IN_ING_SELECAO_VAGA_REMANESC, IN_ING_SELECAO_VAGA_PROG_ESPEC, IN_ING_TRANSF_EXOFFICIO, IN_ING_DECISAO_JUDICIAL, IN_ING_CONVENIO_PECG, IN_RESERVA_VAGAS, IN_RESERVA_ETNICO, IN_RESERVA_DEFICIENCIA, IN_RESERVA_ENSINO_PUBLICO, IN_RESERVA_RENDA_FAMILIAR, IN_RESERVA_OUTRA, IN_FINANC_ESTUDANTIL, IN_FIN_REEMB_FIES, IN_FIN_REEMB_ESTADUAL, IN_FIN_REEMB_MUNICIPAL, IN_FIN_REEMB_PROG_IES, IN_FIN_REEMB_ENT_EXTERNA, IN_FIN_REEMB_OUTRA, IN_FIN_NAOREEMB_PROUNI_INTEGR, IN_FIN_NAOREEMB_PROUNI_PARCIAL, IN_FIN_NAOREEMB_ESTADUAL, IN_FIN_NAOREEMB_MUNICIPAL, IN_FIN_NAOREEMB_PROG_IES, IN_FIN_NAOREEMB_ENT_EXTERNA, IN_FIN_NAOREEMB_OUTRA, IN_APOIO_SOCIAL, IN_APOIO_ALIMENTACAO, IN_APOIO_BOLSA_PERMANENCIA, IN_APOIO_BOLSA_TRABALHO, IN_APOIO_MATERIAL_DIDATICO, IN_APOIO_MORADIA, IN_APOIO_TRANSPORTE, IN_ATIVIDADE_EXTRACURRICULAR, IN_COMPL_ESTAGIO, IN_COMPL_EXTENSAO, IN_COMPL_MONITORIA, IN_COMPL_PESQUISA, IN_BOLSA_ESTAGIO, IN_BOLSA_EXTENSAO, IN_BOLSA_MONITORIA, IN_BOLSA_PESQUISA, CO_TIPO_ESCOLA_ENS_MEDIO, IN_ALUNO_PARFOR, CO_SEMESTRE_CONCLUSAO, CO_SEMESTRE_REFERENCIA, IN_MOBILIDADE_ACADEMICA, CO_MOBILIDADE_ACADEMICA, CO_MOBILIDADE_ACADEMICA_INTERN, CO_IES_DESTINO, CO_PAIS_DESTINO, IN_MATRICULA, IN_CONCLUINTE, IN_INGRESSO_TOTAL, IN_INGRESSO_VAGA_NOVA, ANO_INGRESSO)


In [6]:
# Aplique a função ParseCurso para cada objeto da base
AlunoHeadlessRDD = (AlunoRawData
                    .filter(lambda x: x != header)
                    .map(ParseAluno)
                    )


firstClean = AlunoHeadlessRDD.take(1)[0]
totalRecs = AlunoHeadlessRDD.count()
print firstClean
print totalRecs

Aluno(CO_IES=u'1', NO_IES=u'UNIVERSIDADE FEDERAL DE MATO GROSSO', CO_CATEGORIA_ADMINISTRATIVA=u'1', DS_CATEGORIA_ADMINISTRATIVA=u'P\ufffdblica Federal', CO_ORGANIZACAO_ACADEMICA=u'1', DS_ORGANIZACAO_ACADEMICA=u'Universidade', CO_CURSO=u'1', NO_CURSO=u'DIREITO', CO_CURSO_POLO=u'', CO_TURNO_ALUNO=u'3', DS_TURNO_ALUNO=u'Noturno', CO_GRAU_ACADEMICO=u'1', DS_GRAU_ACADEMICO=u'Bacharelado', CO_MODALIDADE_ENSINO=u'1', DS_MODALIDADE_ENSINO=u'Presencial', CO_NIVEL_ACADEMICO=u'1', DS_NIVEL_ACADEMICO=u'Gradua\ufffd\ufffdo', CO_OCDE=u'380D01', NO_OCDE=u'Direito', CO_OCDE_AREA_GERAL=u'3', NO_OCDE_AREA_GERAL=u'Ci\ufffdncias sociais, neg\ufffdcios e direito', CO_OCDE_AREA_ESPECIFICA=u'38', NO_OCDE_AREA_ESPECIFICA=u'Direito', CO_OCDE_AREA_DETALHADA=u'380', NO_OCDE_AREA_DETALHADA=u'Direito', CO_ALUNO_CURSO=u'25001314', CO_ALUNO_CURSO_ORIGEM=u'', CO_ALUNO=u'400001339152', CO_COR_RACA_ALUNO=u'3', DS_COR_RACA_ALUNO=u'Parda', IN_SEXO_ALUNO=u'0', DS_SEXO_ALUNO=u'Masculino', NU_ANO_ALUNO_NASC=u'1987', NU_MES_

In [9]:
# IES com mais alunos

AlunoIESRDD = (AlunoHeadlessRDD
               .map(lambda rec: (rec.NO_IES,1))
               .reduceByKey(lambda x,y: x+y)
               )
AlunoIESRDDCount = sorted(AlunoIESRDD.collect(), key=lambda x: -x[1])
print '\nAs 20 IES com mais docentes:\n{0}'.format(AlunoIESRDDCount[:20])


As 20 IES com mais docentes:
[(u'UNIVERSIDADE PAULISTA', 542921), (u'UNIVERSIDADE PIT\ufffdGORAS UNOPAR', 514730), (u'UNIVERSIDADE EST\ufffdCIO DE S\ufffd', 417404), (u'UNIVERSIDADE ANHANGUERA - UNIDERP', 306712), (u'CENTRO UNIVERSIT\ufffdRIO INTERNACIONAL', 225553), (u'UNIVERSIDADE NOVE DE JULHO', 217863), (u'CENTRO UNIVERSIT\ufffdRIO LEONARDO DA VINCI', 131301), (u'CENTRO UNIVERSIT\ufffdRIO DAS FACULDADES METROPOLITANAS UNIDAS', 86525), (u'UNIVERSIDADE ANHEMBI MORUMBI', 82734), (u'UNIVERSIDADE ANHANGUERA DE S\ufffdO PAULO - UNIAN-SP', 81860), (u'CENTRO UNIVERSIT\ufffdRIO DE MARING\ufffd - UNICESUMAR', 76242), (u'UNIVERSIDADE DE S\ufffdO PAULO', 70099), (u'UNIVERSIDADE FEDERAL FLUMINENSE', 67588), (u'CENTRO UNIVERSIT\ufffdRIO EST\ufffdCIO DE RIBEIR\ufffdO PRETO', 63896), (u'Centro Universit\ufffdrio Est\ufffdcio do Cear\ufffd', 62032), (u'UNIVERSIDADE CIDADE DE S\ufffdO PAULO', 61067), (u'UNIVERSIDADE CRUZEIRO DO SUL', 61043), (u'UNIVERSIDADE TIRADENTES', 59457), (u'UNIVERSIDADE FEDE

In [10]:
# IES com mais alunos 

AlunoIESRDD = (AlunoHeadlessRDD
               .map(lambda rec: (rec.CO_IES,1))
               .reduceByKey(lambda x,y: x+y)
               )
#AlunoIESRDDCount = sorted(AlunoIESRDD.collect(), key=lambda x: -x[1])
#print '\nAs 20 IES com mais docentes:\n{0}'.format(AlunoIESRDDCount[:20])

print AlunoIESRDD.collect()

[(u'1087', 5021), (u'1043', 17538), (u'1281', 4543), (u'1076', 1198), (u'519', 11204), (u'1843', 700), (u'4000', 1338), (u'2389', 242), (u'506', 806), (u'1425', 380), (u'1858', 232), (u'3643', 125), (u'1086', 171), (u'18067', 301), (u'1280', 230), (u'1042', 23864), (u'1077', 17193), (u'3758', 430), (u'1239', 1735), (u'3039', 99), (u'518', 16105), (u'1842', 1572), (u'17224', 1388), (u'507', 245), (u'657', 685), (u'2397', 411), (u'1041', 12207), (u'16602', 81), (u'1427', 9413), (u'17326', 86), (u'1841', 3454), (u'14028', 2050), (u'1085', 1548), (u'2237', 372), (u'3021', 6), (u'15777', 481), (u'764', 290), (u'2826', 539), (u'4819', 195), (u'3641', 5402), (u'1426', 2632), (u'1075', 4207), (u'2229', 414), (u'14029', 797), (u'1840', 1218), (u'13792', 391), (u'3020', 740), (u'18454', 66), (u'4674', 525), (u'1084', 1722), (u'17226', 1380), (u'5439', 4796), (u'2827', 640), (u'18065', 611), (u'3428', 85), (u'1436', 3901), (u'1609', 186), (u'3232', 171), (u'4079', 641), (u'1429', 387), (u'1847', 

In [7]:
# IES com mais alunos Seqüencial de Formação Específica

AlunoIESRDD = (AlunoHeadlessRDD
               .filter(lambda rec: rec.CO_NIVEL_ACADEMICO == '2')
               .map(lambda rec: (rec.CO_IES,1))
               .reduceByKey(lambda x,y: x+y)
               )


print AlunoIESRDD.collect()

[(u'1041', 206), (u'757', 22), (u'923', 107), (u'466', 636), (u'3303', 240), (u'54', 273), (u'13', 178), (u'386', 139), (u'376', 54), (u'14', 2), (u'446', 4), (u'1255', 3), (u'19', 5), (u'1658', 507), (u'1125', 86), (u'161', 23), (u'847', 9), (u'668', 1), (u'848', 47), (u'4', 41), (u'579', 107), (u'1499', 351), (u'1998', 3), (u'997', 14), (u'409', 40), (u'298', 40), (u'82', 57), (u'316', 5150), (u'701', 31), (u'908', 1), (u'763', 119)]
